# Preprocessing notebook for task 1
- Extracting number of followers of each username helps in estimating the number of likes a tweet would have received.


## Installing and Importing the necessary libraries

In [ ]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Loading the training dataset to a dataframe
 Replace 'behaviour_simulation_train.xlsx' with the appropriate file path of the training dataset.

In [ ]:
df=pd.read_excel('behaviour_simulation_train.xlsx')

## Extracting the Unique Usernames

In [ ]:
df_unique_usernames = pd.DataFrame({'username': df['username'].unique()})

## Function to convert the number of followers text to an integer ( Eg: 1.8k = 1800 )

In [ ]:
def convert_followers_to_int(followers_text):
    followers_text = followers_text.lower()
    if 'k' in followers_text:
        return int(float(followers_text.replace('k', '')) * 1000)
    elif 'm' in followers_text:
        return int(float(followers_text.replace('m', '')) * 1000000)
    else:
        return int(followers_text)

## Function to extract follower count
- This website was used for web-scrapping twitter followers count: https://hypeauditor.com/

In [ ]:
def extract_followers_count(username):
    url = f'https://hypeauditor.com/twitter/{username}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    metrics_div = soup.find('div', {'class': 'metrics flex'})

    if metrics_div:
        metric_divs = metrics_div.find_all('div', {'class': 'metric'})

        for div in metric_divs:
            metric_title = div.find('div', {'class': 'metric-title'})
            if metric_title and metric_title.text.strip() == 'Followers':
                followers_count = div.text.replace('Followers', '').strip()
                return convert_followers_to_int(followers_count)

    return None

## Using the above functions to fetch the follower count of each unique username

In [ ]:
df_unique_usernames['followers'] = df_unique_usernames['username'].apply(lambda x: extract_followers_count(x) if pd.notnull(x) else None)

## A new column "followers" is added to the original dataframe
- A dictionary is created which maps every unique username to the follower count.
- This mapping is used to create a new "followers" column in the original dataframe.

In [ ]:
username_followers_mapping = df_unique_usernames.set_index('username')['followers'].to_dict()
df['followers'] = df['username'].map(username_followers_mapping)

## Converting the dataframe back to an excel sheet

In [ ]:
df.to_excel('PS_8_dataset_with_followers.xlsx')